In [ ]:
import numpy as np
import math

  def fit_conditional_densities(self, X, y):
    self.X_, self.y_ = X, y
    self.n, self.p = self.X_.shape
    if self.ar1:
      self.X_ = self.X_[:, LinearGlucoseModel.AR1_INDICES]
    self.regressor_ = Ridge()
    self.regressor_.fit(self.X_, self.y_)
    self.glucose_sigma_hat = np.sqrt(np.sum((self.regressor_.predict(self.X_) - self.y_)**2) /
                                     (self.n - self.p))
    
  def loglikeli_L_given_trt(self, X_each_patient):
    ## loglikelihood of L_t|trt_{t-1} for each patient
    ## X_each_patient: 2-d array of states for one patient with dim time steps by 7, the last col is actions
    probs = self.prob_L_given_trts[X_each_patient[1:, 0]]
    loglikeli = sum(X_each_patient[1:, 1] * np.log(probs) + (1-X_each_patient[1:, 1]) * np.log(1-probs))
    return loglikeli
  
  def loglikeli_death(self, X_each_patient, y_each_patient):
    x = self.death_prob_coef[0]+self.death_prob_coef[1]*y_each_patient[:-1]**2*(y_each_patient[:-1]>7)+\
        self.death_prob_coef[2]*X_each_patient[:-1, 0]
    prob_c = self.exp_helper(x)
    loglikeli = sum(X_each_patient[1:, 6] * np.log(prob_c) + (1-X_each_patient[1:, 6]) * np.log(1-prob_c))
    return loglikeli
  
  def loglikeli_B_X(self, X_each_patient):
    mean_B = X_each_patient[:-1, 3]/np.sqrt(1+self.sigma_eps**2)
    mean_X = X_each_patient[:-1, 4]/np.sqrt(1+self.sigma_eps**2)
    loglikeli = sum(-np.log(self.sigma_helper(self.sigma_eps))/2 - (X_each_patient[1:, 3]-mean_B)**2/(2*self.sigma_helper(self.sigma_eps)) + \
                    -np.log(self.sigma_helper(self.sigma_eps))/2 - (X_each_patient[1:, 4]-mean_X)**2/(2*self.sigma_helper(self.sigma_eps)))
    return loglikeli

  def A1c_multiply_part_each_patient(self, X_each_patient, y_each_patient):
    K_t = y_each_patient > 7 and X_each_patient[:-1, 0] < 4 and X_each_patient[:-1, 6] != 0 and X_each_patient[1:, 1] != 1
    tau_t = np.zeros(len(K_t))
    tau_t[X_each_patient[1:, 0] != 0] = self.tau_trts[X_each_patient[1:, 0][X_each_patient[1:, 0] != 0]]
    cum_multiply = np.cumprod(1 - K_t * tau_t)
    return cum_multiply * self.mu0

  def loglikeli_A1c(self, X_each_patient, y_each_patient):
    cum_multiply = self.A1c_multiply_part_each_patient(X_each_patient, y_each_patient)
    cond_mean = (y_each_patient[:-1] - cum_multiply[:-1]) / np.sqrt(1+self.sigma_eps**2) + cum_multiply[1:]
    loglikeli = sum(-np.log(self.sigma_helper(self.sigma_eps))/2 - (y_each_patient[1:] - cond_mean)**2 / (2*self.sigma_helper(self.sigma_eps)))
    return loglikeli

  def joint_loglikeli(self):
    # taus: paramenters of treatment effect
    # the function need to be miniized
    log_likelihood = 0
    for i in range(self.nPatients):
      X_each_patient, y_each_patient = self.X_[i], self.y_[i]
      log_likelihood += self.loglikeli_L_given_trt(X_each_patient, y_each_patient) + \
                        self.loglikeli_death(X_each_patient, y_each_patient) + \
                        self.loglikeli_A1c(X_each_patient, y_each_patient) + \
                        self.loglikeli_B_X(X_each_patient) + \
                        -np.log(self.sigma_B0_X0_Y0)-(X_each_patient[0,3]-self.mu_B0_X0)**2/(2*self.sigma_B0_X0_Y0**2) + \
                        -np.log(self.sigma_B0_X0_Y0)-(X_each_patient[0,4]-self.mu_B0_X0)**2/(2*self.sigma_B0_X0_Y0**2) + \
                        -np.log(self.sigma_B0_X0_Y0)-(y_each_patient[0]-self.mu0)**2/(2*self.sigma_B0_X0_Y0**2)
    return log_likelihood
        
  def exp_helper(self, x):
    return float(math.exp(x))/float(1+math.exp(x))
  
  def sigma_helper(self, sigma);
    return float(sigma**2/(1+sigma**2))